In [3]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [4]:
# Inputs
parcels = r".\Inputs\WFRC_Non_Residential_Parcels.gdb\Non_Residential_Parcels"
hui = r"E:\Projects\Housing-Unit-Inventory\Current_Version\wfrc_hui_20220106.gdb\wfrc_housing_unit_inventory_20220106"
mag_parcels = r'.\Inputs\MAG_REMM_Parcels_for_Splitting.gdb\MAG_REMM_Parcels_for_splitting'
raw_parcel_pts = r'.\Inputs\parcel_centroids.gdb\wfrc_centroids'
addr_pts = r'.\Inputs\addr_pts.gdb\addr_pts_wfrc_mag'
new_taz = r".\Inputs\New_TAZ.shp"

In [5]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "recompile3.gdb", 'remm_base_year_2019.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [6]:
hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
hui_sdf = hui_sdf[['TYPE', 'IS_OUG', 'COUNTY', 'UNIT_COUNT', 'DUA', 'APX_HGHT','ACRES', 'TOT_BD_FT2', 'TOT_VALUE', 'APX_BLT_YR', 'UNIT_ID', 'SHAPE']].copy()
hui_sdf.columns = ['TYPE', 'IS_OUG', 'COUNTY_NAME', 'UNIT_COUNT', 'DUA', 'FLOORS_CNT','PARCEL_ACRES', 'BLDG_SQFT', 'TOT_MKT_VALUE', 'BUILT_YR','UNIT_ID', 'SHAPE']
hui_copy =  os.path.join(gdb, '_00_hui_copy')
hui_sdf.spatial.to_featureclass(location=hui_copy,sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\recompile3.gdb\\_00_hui_copy'

In [7]:
# copy hui
# hui_copy = arcpy.FeatureClassToFeatureClass_conversion(hui, gdb, '_00_hui_copy')
arcpy.CalculateField_management(hui_copy, "TOTAL_MKT_VALUE", """0""", "PYTHON3")

# use spatial join to summarize TOTAL_MKT_VALUE, LAND_MKT_VALUE, BLDG_SQFT
target_features = hui
join_features = raw_parcel_pts
output_features = os.path.join(gdb, "_00_hui_pc_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total land value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building square feet
fieldindex = fieldmappings.findFieldMapIndex('BLDG_SQFT')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

In [8]:
# post process hui
sj_sdf = pd.DataFrame.spatial.from_featureclass(sj[0])
sj_sdf = sj_sdf[['UNIT_ID', 'TOTAL_MKT_VALUE','LAND_MKT_VALUE','BLDG_SQFT', 'COUNTY_NAME']].copy()
sj_sdf.columns = ['UNIT_ID', 'TOTAL_MKT_VALUE_NEW','LAND_MKT_VALUE','BLDG_SQFT_NEW', 'COUNTY_NAME']

# hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
merged = hui_sdf.merge(sj_sdf, left_on=['UNIT_ID','COUNTY_NAME'],right_on=['UNIT_ID','COUNTY_NAME'],how='left')

merged['PARCEL_ACRES'] = pd.to_numeric(merged['PARCEL_ACRES'].round(4)) 

# recalc the building type
merged.loc[(merged['TYPE'] == 'single_family'), 'building_type_id'] = 1
merged.loc[(merged['TYPE'] == 'multi_family'), 'building_type_id'] = 2

# if current value is zero, update
# merged.loc[((merged['TOTAL_MKT_VALUE'] == 0) | (merged['TOTAL_MKT_VALUE'].isnull() == True)) & 
#             (merged['TOTAL_MKT_VALUE_NEW'] != 0), 'TOTAL_MKT_VALUE'] = merged['TOTAL_MKT_VALUE_NEW']

merged['TOTAL_MKT_VALUE'] =  merged['TOTAL_MKT_VALUE_NEW']

merged.loc[((merged['BLDG_SQFT'] == 0) | (merged['BLDG_SQFT'].isnull() == True)) & 
            (merged['BLDG_SQFT_NEW'] != 0), 'BLDG_SQFT'] = merged['BLDG_SQFT_NEW']

hui_post_process = merged.spatial.to_featureclass(location=os.path.join(gdb, "_01_hui_post_process"),sanitize_columns=False)

In [9]:
# select all parcels within buffered housing unit inventory parcels and delete
hui_buff = arcpy.Buffer_analysis(hui, os.path.join(gdb, "_02_hui_buffer_1_25m"), '1.25 METERS')

In [10]:
# select and remove existing parcels
parcels_copy = arcpy.FeatureClassToFeatureClass_conversion(parcels, gdb, '_03_parcels_copy')
parcels_with_selection = arcpy.SelectLayerByLocation_management(parcels_copy, 'WITHIN', hui_buff)
arcpy.DeleteFeatures_management(parcels_with_selection)

<Result '_03_parcels_copy_Layer2'>

In [11]:
# add reforamtted hui parcels to main parcel dataset
parcels_reunited = arcpy.Merge_management([parcels_with_selection, hui_post_process], os.path.join(gdb, '_04_WFRC_Reunited_Parcels_'))

# Eliminate WFRC Road Polygons


In [12]:
parcels_reunited_lyr = arcpy.MakeFeatureLayer_management(parcels_reunited, 'parcels_split_merge_lyr') 
arcpy.AddField_management(parcels_reunited_lyr, 'Thinness', 'FLOAT')
arcpy.CalculateField_management(parcels_reunited_lyr, field='Thinness', expression="(4 * math.pi* !Shape_Area!) / (!Shape_Length! * !Shape_Length!)", expression_type="PYTHON3")
query = """(Thinness < 0.26 And building_type_id NOT IN (1, 2,3, 5, 4, 10, 3, 7, 8,9,10,11,12,13, 14, 15)) And PARCEL_ACRES > 0.025 And PARCEL_ID NOT IN ('22081850140000', '15011090062000','110810001') And agriculture <> 1"""

arcpy.SelectLayerByAttribute_management(parcels_reunited_lyr, 'NEW_SELECTION', query)

# exclude parcels that overlap with pts from selection
do_not_delete = '.\inputs\do_not_delete_thinness.shp'
arcpy.SelectLayerByLocation_management(in_layer=parcels_reunited_lyr, overlap_type="INTERSECT",
                                       select_features=do_not_delete,
                                       selection_type='REMOVE_FROM_SELECTION')

parcels_reunited_lyr = arcpy.DeleteFeatures_management(parcels_reunited_lyr)

# Merge with MAG Parcels

In [13]:
# standardize
mag_parcels_sdf = pd.DataFrame.spatial.from_featureclass(mag_parcels)
mag_parcels_sdf.loc[(mag_parcels_sdf['OUG_ID']>0), 'IS_OUG'] = '1'
mag_parcels_sdf.loc[(mag_parcels_sdf['OUG_ID']==0), 'IS_OUG'] = '0'
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"LAND_VAL": "LAND_MKT_VALUE"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"TOT_VAL": "TOTAL_MKT_VALUE"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"residential_units": "UNIT_COUNT"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"building_sqft": "BLDG_SQFT"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"year_built": "BUILT_YR"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"ACREAGE": "PARCEL_ACRES"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"Parent_Parcel": "PARCEL_ID"})
mag_parcels_sdf = mag_parcels_sdf.rename(columns={"parcel_id_REMM": "parcel_id_remm"})
mag_parcels_sdf['PARCEL_ID'] = mag_parcels_sdf['PARCEL_ID'].astype(str)

mag_parcels_sdf['COUNTY_NAME'] = 'Utah'
mag_parcels_sdf['COUNTY_ID'] = 49

# set parcels built after 2019 to empty
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'TOTAL_MKT_VALUE'] = mag_parcels_sdf['LAND_MKT_VALUE']
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'building_type_id'] = 0
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'UNIT_COUNT'] = 0
mag_parcels_sdf.loc[(mag_parcels_sdf['BUILT_YR']>2019), 'BLDG_SQFT'] = 0

mag_parcels_sdf = mag_parcels_sdf[['COUNTY_NAME','COUNTY_ID',"TOTAL_MKT_VALUE",'IS_OUG','building_type_id',"LAND_MKT_VALUE", "UNIT_COUNT","BLDG_SQFT","BUILT_YR","PARCEL_ACRES","PARCEL_ID", 'parcel_id_remm', "SHAPE"]].copy()

mag_parcels_sdf.spatial.to_featureclass(location=os.path.join(gdb, "_04_mag_parcels_processed"),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\recompile3.gdb\\_04_mag_parcels_processed'

In [14]:
# wfrc_parcels_sdf = pd.DataFrame.spatial.from_featureclass(parcels_reunited[0])
# wfrc_parcels_sdf = wfrc_parcels_sdf.rename(columns={"parcel_id_remm": "parcel_id_REMM"})

# # stack parcels and create remm paarcel id
# wfrc_mag_parcels_sdf = pd.concat([wfrc_parcels_sdf, mag_parcels_sdf])
# wfrc_mag_parcels_sdf['parcel_id_REMM'] = wfrc_mag_parcels_sdf.index

# wfrc_mag_parcels = os.path.join(gdb, '_05_wfrc_mag_parcels')
# wfrc_mag_parcels_sdf.spatial.to_featureclass(location=wfrc_mag_parcels,sanitize_columns=False)

wfrc_mag_parcels = arcpy.Merge_management([parcels_reunited_lyr, os.path.join(gdb, "_04_mag_parcels_processed")], os.path.join(gdb, '_05_wfrc_mag_parcels'))

code = """rec=0 
def autoIncrement(): 
 global rec 
 pStart = 1  
 pInterval = 1 
 if (rec == 0):  
  rec = pStart  
 else:  
  rec += pInterval  
 return rec"""

arcpy.CalculateField_management(parcels_reunited_lyr, 'parcel_id_remm', """autoIncrement()""", "PYTHON3", 
                                code)


<Result 'parcels_split_merge_lyr'>

In [15]:
# clear some memory
del hui_sdf
del merged
# del wfrc_parcels_sdf
del mag_parcels_sdf
# del wfrc_mag_parcels_sdf
del parcels_reunited_lyr
del parcels_reunited

# Split Parcels

In [16]:
env.outputCoordinateSystem = arcpy.SpatialReference('NAD 1983 UTM Zone 12N')
desc = arcpy.Describe(wfrc_mag_parcels)

#create the acre mesh- in this case, the cell_width and height calculate to 1 acre.
acre_mesh = arcpy.management.CreateFishnet(out_feature_class = os.path.join(gdb,'_05a_acre_mesh'),
    origin_coord = str(desc.extent.lowerLeft),
    y_axis_coord = str(desc.extent.XMin) + " " + str(desc.extent.YMax + 10),
    cell_width = 63.6, # meters
    cell_height = 63.6, # meters
    number_rows = '#',
    number_columns = '#',
    labels = 'NO_LABELS', 
    template = wfrc_mag_parcels, 
    geometry_type = 'POLYGON'
)

#Add Required Fields: RealBuildingID
arcpy.management.AddFields(in_table=acre_mesh, field_description=[["RealBldg", "DOUBLE", "RealBldg", "", "", ""]])


<Result '.\\Outputs\\recompile3.gdb\\_05a_acre_mesh'>

In [17]:
#Identifying which parcels meet the qualifications
Parcels_7Acre_wBuildings = arcpy.conversion.FeatureClassToFeatureClass(in_features=wfrc_mag_parcels, 
                                                                       out_path=gdb, out_name="_06_Parcels_7Acre_wBuildings", 
                                                                       where_clause="(building_type_id IN (0, 1, 15)) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1')")

#Identifying which buildings reside on those parcels that meet qualifications
Buildings_7Acre_wBuildings = arcpy.analysis.SpatialJoin(target_features=addr_pts, 
                                                        join_features=Parcels_7Acre_wBuildings, 
                                                        out_feature_class=os.path.join(gdb,'_07_Buildings_7Acre_wBuildings'), 
                                                        join_operation="JOIN_ONE_TO_ONE", 
                                                        join_type="KEEP_COMMON",
                                                        match_option="INTERSECT", 
                                                        search_radius="", 
                                                        distance_field_name="")

#Adding the Preserve Bldg field- used to identify which building areas to preserve
Buildings_7Acre_wBuildings = arcpy.management.AddField(in_table=Buildings_7Acre_wBuildings, 
                                                       field_name="PreserveBldg", 
                                                       field_type="SHORT", 
                                                       field_precision=None, 
                                                       field_scale=None, 
                                                       field_length=None, 
                                                       field_alias="RealBldg", 
                                                       field_is_nullable="NULLABLE", 
                                                       field_is_required="NON_REQUIRED", 
                                                       field_domain="")

Buildings_7Acre_wBuildings= arcpy.management.CalculateField(in_table=Buildings_7Acre_wBuildings, 
                                                    field="PreserveBldg", 
                                                    expression="1", 
                                                    expression_type="ARCADE", 
                                                    code_block="", 
                                                    field_type="TEXT")

Preserve_Buildings = arcpy.conversion.FeatureClassToFeatureClass(in_features=Buildings_7Acre_wBuildings, 
                                                                       out_path=gdb, out_name="_08_Preserve_Buildings")

In [18]:
#Use Spatial Join to identify which acre_mesh pieces contain those preserve_buildings
Mesh_w_Buildings = arcpy.analysis.SpatialJoin(target_features=acre_mesh, 
                           join_features=Preserve_Buildings, 
                           out_feature_class=os.path.join(gdb,'_09_Mesh_w_Buildings'), 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL",
                           match_option="INTERSECT", 
                           search_radius="", 
                           distance_field_name="")

#convert to new Feature Class if Preserve_blg = 1
Mesh_w_PreserveBldgs = arcpy.conversion.FeatureClassToFeatureClass(in_features=Mesh_w_Buildings, 
                                                                       out_path=gdb, out_name="_10_Mesh_w_PreserveBldgs", 
                                                                       where_clause="PreserveBldg IN (1)")

In [19]:
#Select the parcels with buildings on them and prep for mesh erasure and convert to feature class
Parcels_for_Erase_select = arcpy.management.SelectLayerByLocation(in_layer=[wfrc_mag_parcels],
                                                                 overlap_type="INTERSECT", 
                                                                 select_features=Buildings_7Acre_wBuildings, 
                                                                 search_distance="", 
                                                                 selection_type="NEW_SELECTION", 
                                                                 invert_spatial_relationship="NOT_INVERT")

Parcels_for_Erase= arcpy.conversion.FeatureClassToFeatureClass(in_features=Parcels_for_Erase_select, 
                                                                       out_path=gdb, out_name="_11_Parcels_for_Erase")

In [20]:
parcels_erased = arcpy.analysis.Erase(in_features=Parcels_for_Erase,
                                      erase_features=Mesh_w_PreserveBldgs, 
                                      out_feature_class= os.path.join(gdb,'_12_parcels_erased'),
                                      cluster_tolerance="")

In [21]:
# calculate all building parcel attributes to zero out for the non-building pieces
#YEAR BUILT
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="BUILT_YR", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

#BUILDING SQFT
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="BLDG_SQFT", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

#BUILDING TYPE ID
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="building_type_id", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

In [22]:
#join erased parcels to original parcels- puzzle piece 1
parcels_with_structures = arcpy.analysis.SymDiff(Parcels_for_Erase, parcels_erased, os.path.join(gdb,'_13_parcels_with_structures'))
arcpy.AddField_management(parcels_with_structures, 'Split', 'LONG')
arcpy.CalculateField_management(parcels_with_structures, field='Split', expression=2, expression_type="PYTHON3") 

parcels_erased_union = arcpy.management.Merge(inputs=[parcels_erased, parcels_with_structures],
                                           output=os.path.join(gdb,'_14_parcels_erased_union'),
                                           add_source="NO_SOURCE_INFO")

In [23]:
#All other parcels not included in the parcels_erased_union
Other_parcels_select = arcpy.management.SelectLayerByLocation(in_layer=[wfrc_mag_parcels], 
                                                          overlap_type="INTERSECT", 
                                                          select_features=Buildings_7Acre_wBuildings, 
                                                          search_distance="", 
                                                          selection_type="NEW_SELECTION", 
                                                          invert_spatial_relationship="INVERT")
#convert selection to new feature class
All_other_parcels= arcpy.conversion.FeatureClassToFeatureClass(in_features=Other_parcels_select, 
                                                                       out_path=gdb, 
                                                               out_name="_15_All_other_parcels")

In [24]:
#merging the non-erased, and the erased parcels back together
prepared_parcels = arcpy.management.Merge(inputs=[All_other_parcels, parcels_erased_union],
                                           output=os.path.join(gdb,'_16_prepared_parcels'),
                                           add_source="NO_SOURCE_INFO")

In [25]:
#select parcels with acreage larger than 7 or haven't been erased
parcels_YES_split_select = arcpy.management.SelectLayerByAttribute(in_layer_or_view=prepared_parcels,
                                                                               selection_type="NEW_SELECTION", 
                                                                               where_clause=""" building_type_id IN (0, 1, 15) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1') And 
                                                                                               (FID__12_parcels_erased > -1 Or FID__12_parcels_erased IS NULL)""", 
                                                                               invert_where_clause="")

parcels_YES_split = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_YES_split_select, 
                                                                       out_path=gdb, 
                                                               out_name="_17_parcels_YES_split")

arcpy.CalculateField_management(parcels_YES_split, field='Split', expression=1, expression_type="PYTHON3") 

# WRONG PLACE apply taz to splitting process
taz_layer = arcpy.MakeFeatureLayer_management(new_taz, 'new_taz') 
query = '''REMM IN (1)'''
arcpy.management.SelectLayerByAttribute(taz_layer, "NEW_SELECTION", query)
parcels_YES_split_taz = arcpy.Identity_analysis(parcels_YES_split, taz_layer, os.path.join(gdb,'_18_parcels_YES_split_taz'))

In [26]:
parcels_split = arcpy.management.SubdividePolygon(in_polygons=parcels_YES_split_taz, 
                                  out_feature_class= os.path.join(gdb,'_19_parcels_split'), 
                                  method="EQUAL_AREAS", 
                                  num_areas=None, 
                                  target_area="5 Acres", 
                                  target_width="", 
                                  split_angle=0, 
                                  subdivision_type="STACKED_BLOCKS")

In [27]:
#select all other parcels that arent ready for splitting
parcels_NO_split_select = arcpy.management.SelectLayerByAttribute(in_layer_or_view=prepared_parcels,
                                                                               selection_type="NEW_SELECTION", 
                                                                               where_clause="""building_type_id IN (0, 1, 15) And (PARCEL_ACRES >= 7) AND (IS_OUG <>  '1')
                                                                                              And (FID__12_parcels_erased > -1 Or FID__12_parcels_erased IS NULL)""", 
                                                                               invert_where_clause="INVERT")


parcels_NO_split = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_NO_split_select, 
                                                                       out_path=gdb, 
                                                               out_name="_20_parcels_NO_split")

In [28]:
#merging all parcels back together again
parcels_split_merge = arcpy.management.Merge(inputs=[parcels_split, parcels_NO_split],
                                           output=os.path.join(gdb,'_21_parcels_split_merge'),
                                           add_source="NO_SOURCE_INFO")

# recalc acreage
arcpy.AddField_management(parcels_split_merge, 'PARCEL_ACRES_NEW', 'FLOAT')
arcpy.CalculateField_management(parcels_split_merge, "PARCEL_ACRES_NEW", """!SHAPE.area@ACRES!""", "PYTHON3")

# create layer
parcels_split_merge_lyr = arcpy.MakeFeatureLayer_management(parcels_split_merge, 'parcels_split_merge_lyr') 

In [29]:
# recalc gflu
# set land use type on parcels using glfu
gflu = '.\Inputs\FutureLandUse2020.gdb\FutureLandUse2020'
gflu_lyr = arcpy.MakeFeatureLayer_management(gflu, 'gflu_lyr') 


##################
# Type 1 (SF)
##################

# query for land use type
query = """ GenLUType IN ('Any Development','Any Residential', 'Mixed Use SF', 'Residential SF', 'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type1', expression="'t'",
                                expression_type="PYTHON3")


##################
# Type 2 (MF)
##################

# query for land use type
query = """ GenLUType IN ('Any Residential', 'Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 
                          'Residential MF/Office', 'Residential MF', 'Mixed Use MF', 'Mixed Use SF', 'Residential/Office', 
                          'Residential/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type2', expression="'t'",
                                expression_type="PYTHON3")

#######################
# Type 3 (Industrial)
#######################

# query for land use type
query = """ GenLUType IN ('Industrial', 'Industrial/Office', 'Any Development', 'Industrial/Mixed Use MF') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type3', expression="'t'",
                                expression_type="PYTHON3")

#######################
# Type 4 (Retail)
#######################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type4', expression="'t'",
                                expression_type="PYTHON3")


#######################
# Type 5 (Office)
#######################

# query for land use type
query = """ GenLUType IN ('Retail/Office', 'Office', 'Any Commercial', 'Residential/Office', 'Residential MF/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type5', expression="'t'",
                                expression_type="PYTHON3")

####################################
# Type 6 (Government and Education)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Government/Education') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type6', expression="'t'",
                                expression_type="PYTHON3")


####################################
# Type 7 (Mixed Use)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 'Mixed Use MF', 'Mixed Use SF', 
                          'Residential MF/Office', 'Residential SF/Retail', 'Residential/Office', 'Residential/Retail', 
                          'Retail/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(parcels_split_merge_lyr, field='type7', expression="'t'",
                                expression_type="PYTHON3")

####################################
# Type 8 (Other)
###################################

# # query for land use type
# query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
#                           'Residential SF/Retail') """
# arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# # select the parcels
# arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
#                                        select_features=gflu_lyr,
#                                        selection_type='NEW_SELECTION')

# arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'t'".format(tag),
#                                 expression_type="PYTHON3")

#################################
# Undevelopable
#################################

# query for land use type
query = """ GenLUType IN ('NoBuild') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_split_merge_lyr, overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

query = """ NoBuild = 1 """
arcpy.SelectLayerByAttribute_management(parcels_split_merge_lyr, 'NEW_SELECTION', query)


# set all types to false
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type1', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type2', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type3', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type4', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type5', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type6', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type7', expression="'f'",
                                expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_split_merge_lyr, field='type8', expression="'f'",
                                expression_type="PYTHON3")

# clear selected parcels
arcpy.SelectLayerByAttribute_management(parcels_split_merge_lyr, 'CLEAR_SELECTION', query)
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'CLEAR_SELECTION', query)
    
    
    
    
# reformat fields to match with REMM (building type)

<Result 'gflu_lyr'>

In [30]:
#attribute cleanup
final_parcel_df = pd.DataFrame.spatial.from_featureclass(parcels_split_merge[0])
#list(final_parcel_df.columns)

# Adjust non-true zoning to false
final_parcel_df.loc[(final_parcel_df['type1'] != 't'), 'type1'] = 'f'
final_parcel_df.loc[(final_parcel_df['type2'] != 't'), 'type2'] = 'f'
final_parcel_df.loc[(final_parcel_df['type3'] != 't'), 'type3'] = 'f'
final_parcel_df.loc[(final_parcel_df['type4'] != 't'), 'type4'] = 'f'
final_parcel_df.loc[(final_parcel_df['type5'] != 't'), 'type5'] = 'f'
final_parcel_df.loc[(final_parcel_df['type6'] != 't'), 'type6'] = 'f'
final_parcel_df.loc[(final_parcel_df['type7'] != 't'), 'type7'] = 'f'
final_parcel_df.loc[(final_parcel_df['type8'] != 't'), 'type8'] = 'f'

# divide land value by this number for new approximate value
final_parcel_df['Split_Factor'] = final_parcel_df['PARCEL_ACRES']/final_parcel_df['PARCEL_ACRES_NEW']
final_parcel_df['PARCEL_ACRES'] = final_parcel_df['PARCEL_ACRES_NEW']

fields = ['COUNTY_NAME', 'COUNTY_ID', 'PARCEL_ID', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'UNIT_COUNT', 'BLDG_SQFT', 'FLOORS_CNT',
         'BUILT_YR', 'EFFBUILT_YR', 'IS_OUG','max_dua', 'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5', 'type6', 
         'type7','type8', 'agriculture', 'basebldg', 'NoBuild', 'redev_friction', 'building_type_id', 'x', 'y', 
         'parcel_id_remm', 'PARCEL_ACRES', 'Split','Split_Factor', 'SHAPE']


# save old parcel id
final_parcel_df = final_parcel_df[fields].copy()
final_parcel_df['parcel_id_REMM_old'] = final_parcel_df['parcel_id_remm']

# generate unique building id for split parcels
final_parcel_df.loc[(final_parcel_df['Split'].isna() == True), 'Split'] = 0
split_parcels = final_parcel_df[final_parcel_df['Split'] == 1].copy()
nonsplit_parcels = final_parcel_df[(final_parcel_df['Split']==0) | (final_parcel_df['Split']==2)].copy()
count = final_parcel_df.shape[0]
split_parcels['parcel_id_remm'] = count + split_parcels.index + 1

final_parcel_df = pd.concat([nonsplit_parcels, split_parcels])

# set residential unit count of new parcels resulting from split to 0
final_parcel_df.loc[(final_parcel_df['Split']==1), 'UNIT_COUNT'] = 0

# recalc land value and total market value using split factor
final_parcel_df.loc[(final_parcel_df['Split']==1), 'LAND_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor']
final_parcel_df.loc[(final_parcel_df['Split']==1), 'TOTAL_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE']
final_parcel_df.loc[(final_parcel_df['Split']==2), 'TOTAL_MKT_VALUE'] = (final_parcel_df['TOTAL_MKT_VALUE'] - final_parcel_df['LAND_MKT_VALUE']) + (final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor'])
final_parcel_df.loc[(final_parcel_df['Split']==2), 'LAND_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor']

In [31]:
# fix county ids
final_parcel_df.loc[final_parcel_df['COUNTY_NAME'] == 'Davis', 'COUNTY_ID'] = '11'
final_parcel_df.loc[final_parcel_df['COUNTY_NAME'] == 'Salt Lake', 'COUNTY_ID'] = '35'
final_parcel_df.loc[final_parcel_df['COUNTY_NAME'] == 'Weber', 'COUNTY_ID'] = '57'

# add type column
final_parcel_df.loc[(final_parcel_df['building_type_id']==0), 'TYPE'] = 'Empty Buildable'
final_parcel_df.loc[(final_parcel_df['building_type_id']==1), 'TYPE'] = 'Single Family Res'
final_parcel_df.loc[(final_parcel_df['building_type_id']==2), 'TYPE'] = 'Multi Family Res'
final_parcel_df.loc[(final_parcel_df['building_type_id']==3), 'TYPE'] = 'Industrial'
final_parcel_df.loc[(final_parcel_df['building_type_id']==4), 'TYPE'] = 'Retail'
final_parcel_df.loc[(final_parcel_df['building_type_id']==5), 'TYPE'] = 'Office'
final_parcel_df.loc[(final_parcel_df['building_type_id']==6), 'TYPE'] = 'Governmental'
final_parcel_df.loc[(final_parcel_df['building_type_id']==7), 'TYPE'] = 'Mixed Use'
final_parcel_df.loc[(final_parcel_df['building_type_id']==8), 'TYPE'] = 'Other'
final_parcel_df.loc[(final_parcel_df['building_type_id']==9), 'TYPE'] = 'Educational'
final_parcel_df.loc[(final_parcel_df['building_type_id']==10), 'TYPE'] = 'Church'
final_parcel_df.loc[(final_parcel_df['building_type_id']==11), 'TYPE'] = 'Group Quarters'
final_parcel_df.loc[(final_parcel_df['building_type_id']==12), 'TYPE'] = 'Commons'
final_parcel_df.loc[(final_parcel_df['building_type_id']==13), 'TYPE'] = 'Healthcare'
final_parcel_df.loc[(final_parcel_df['building_type_id']==14), 'TYPE'] = 'Open Space Non-Buildable'
final_parcel_df.loc[(final_parcel_df['building_type_id']==15), 'TYPE'] = 'Agriculture'
final_parcel_df.loc[(final_parcel_df['building_type_id']==16), 'TYPE'] = 'Utilities'
final_parcel_df.loc[(final_parcel_df['building_type_id']==99), 'TYPE'] = 'No Build'

# final_parcel_df['BLT_DECADE'] = 'NA'
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1840) & (final_parcel_df['BUILT_YR'] < 1850), 'BLT_DECADE'] = "1840's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1850) & (final_parcel_df['BUILT_YR'] < 1860), 'BLT_DECADE'] = "1850's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1860) & (final_parcel_df['BUILT_YR'] < 1870), 'BLT_DECADE'] = "1860's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1870) & (final_parcel_df['BUILT_YR'] < 1880), 'BLT_DECADE'] = "1870's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1880) & (final_parcel_df['BUILT_YR'] < 1890), 'BLT_DECADE'] = "1880's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1890) & (final_parcel_df['BUILT_YR'] < 1900), 'BLT_DECADE'] = "1890's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1900) & (final_parcel_df['BUILT_YR'] < 1910), 'BLT_DECADE'] = "1900's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1910) & (final_parcel_df['BUILT_YR'] < 1920), 'BLT_DECADE'] = "1910's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1920) & (final_parcel_df['BUILT_YR'] < 1930), 'BLT_DECADE'] = "1920's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1930) & (final_parcel_df['BUILT_YR'] < 1940), 'BLT_DECADE'] = "1930's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1940) & (final_parcel_df['BUILT_YR'] < 1950), 'BLT_DECADE'] = "1940's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1950) & (final_parcel_df['BUILT_YR'] < 1960), 'BLT_DECADE'] = "1950's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1960) & (final_parcel_df['BUILT_YR'] < 1970), 'BLT_DECADE'] = "1960's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1970) & (final_parcel_df['BUILT_YR'] < 1980), 'BLT_DECADE'] = "1970's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1980) & (final_parcel_df['BUILT_YR'] < 1990), 'BLT_DECADE'] = "1980's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 1990) & (final_parcel_df['BUILT_YR'] < 2000), 'BLT_DECADE'] = "1990's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 2000) & (final_parcel_df['BUILT_YR'] < 2010), 'BLT_DECADE'] = "2000's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 2010) & (final_parcel_df['BUILT_YR'] < 2020), 'BLT_DECADE'] = "2010's"
# final_parcel_df.loc[(final_parcel_df['BUILT_YR'] >= 2020) & (final_parcel_df['BUILT_YR'] < 2030), 'BLT_DECADE'] = "2020's"

# make fields REMM compatible
final_parcel_df = final_parcel_df.rename(columns={"LAND_MKT_VALUE": "land_value"})
final_parcel_df.loc[(final_parcel_df['building_type_id'].isin([1,2])), 'residential_price'] = final_parcel_df['TOTAL_MKT_VALUE']
final_parcel_df.loc[~(final_parcel_df['building_type_id'].isin([1,2])), 'non_residential_price'] = final_parcel_df['TOTAL_MKT_VALUE']
final_parcel_df = final_parcel_df.rename(columns={"COUNTY_NAME": "CO_NAME"})
final_parcel_df = final_parcel_df.rename(columns={"BLDG_SQFT": "building_sqft"})
final_parcel_df.loc[~(final_parcel_df['building_type_id'].isin([1,2])), 'non_residential_sqft'] = final_parcel_df['building_sqft']
final_parcel_df = final_parcel_df.rename(columns={"UNIT_COUNT": "residential_units"})
final_parcel_df = final_parcel_df.rename(columns={"BUILT_YR": "year_built"})
final_parcel_df = final_parcel_df.rename(columns={"PARCEL_ACRES": "parcel_acres"})
final_parcel_df = final_parcel_df.rename(columns={"TYPE": "building_type"})
final_parcel_df = final_parcel_df.rename(columns={"parcel_id_remm": "parcel_id_REMM"})

In [32]:
# final export
final_parcel_df.spatial.to_featureclass(location=os.path.join(gdb2,'parcels_2019'), sanitize_columns=False)


'e:\\Projects\\REMM-Input-Data-Prep-2019\\Parcels\\2020-WFRC\\Outputs\\remm_base_year_2019.gdb\\parcels_2019'

In [33]:
# import subprocess
os.startfile(r".\Review.aprx")